# XML to Python

17 Mar: Nichy stop at Step 4. Tim comes to help on IOB part.

19 Mar: Nichy finished step 4 (converting to IOB format). Time is helping on extracting misspelling words.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Get 'text', 'aspectTerm' and 'aspectCategory from XML

The bold letters are what we get. I will use the same variable names as in XML, so we don't get confused.


\<text\>**But the staff was so horrible to us.**\<\/text\>

\<aspectTerm **term="staff" polarity="negative"** from="8" to="13"\/\>

\<aspectCategory **category="service" polarity="negative**"\/\>


In [27]:
!ls '/content/gdrive/MyDrive/Colab Notebooks/CS4248 NLP Project/'

'Instruction to use Colab.ipynb'	 pre-process.ipynb
'IOB Preprocessing.ipynb'		 Restaurants_Test.xml
 pre_processed.csv			 Restaurants_Train.xml
'Preprocessing xml to dataframe.ipynb'	'Unsupervised Model.ipynb'


In [2]:
import xml.etree.ElementTree as ET

restaurant = ET.parse('/content/gdrive/MyDrive/Colab Notebooks/CS4248 NLP Project/Restaurants_Train.xml')
sentences = restaurant.getroot()

# List: a list of 'text' or a sentence from XML.
texts = []
# List: a list of 'term' in each text
terms = []
# List: a list of 'category' in each text
categories = []

# Dictionary: {term: number of terms founded in all texts}
term_positive = {}
term_negative = {}
term_neutral  = {}
term_conflict = {}

# Dictionary: {category: number of category founded in all texts}
category_positive = {}
category_negative = {}
category_neutral  = {}
category_conflict = {}


for sentence in sentences:
    hasTerm, hasCategory = False, False
    for taa in sentence: # taa is the level of <text>, <aspectTerms>, <aspectCategories>
        if(taa.tag == 'text'):
            texts.append(taa.text)
            #print(sentence.tag, taa.tag, ':', taa.text)
        elif(taa.tag == 'aspectTerms'): 
            hasTerm = True
            termInEachText = []
            for aspectTerms in taa:
                #print(sentence.tag, taa.tag, ':', aspectTerms.attrib) # attrib is dictionary
                term = aspectTerms.attrib['term']
                polarity = aspectTerms.attrib['polarity']
                termInEachText.append(term)
            if(polarity == 'positive'):
                term_positive[term] = term_positive.get(term, 0) + 1
            elif(polarity == 'negative'):
                term_negative[term] = term_negative.get(term, 0) + 1
            elif(polarity == 'neutral'):
                term_neutral[term] = term_neutral.get(term, 0) + 1
            elif(polarity == 'conflict'):
                term_conflict[term] = term_conflict.get(term, 0) + 1
        elif(taa.tag == 'aspectCategories'): 
            hasCategory = True
            categoryInEachText = []
            for aspectCategories in taa:
                #print(sentence.tag, taa.tag, ':', aspectCategories.attrib)
                category = aspectCategories.attrib['category']
                polarity = aspectCategories.attrib['polarity']
                categoryInEachText.append(category)
            #categories.append(categoryInEachText)
            if(polarity == 'positive'):
                category_positive[category] = category_positive.get(category, 0) + 1
            elif(polarity == 'negative'):
                category_negative[category] = category_negative.get(category, 0) + 1
            elif(polarity == 'neutral'):
                category_neutral[category] = category_neutral.get(category, 0) + 1
            elif(polarity == 'conflict'):
                category_conflict[category] = category_conflict.get(category, 0) + 1
                
    if(hasTerm == True):
        terms.append(termInEachText)
    elif(hasTerm == False):
        terms.append([])
    
    if(hasCategory == True):
        categories.append(categoryInEachText)
    elif(hasCategory == False):
        categories.append([])

## Print to view the result

Currently we will use **texts** for preprocessing to make texts in IOB format. The remaining dictionary is for further use.

In [ ]:
"""
# List: a list of 'text' or a sentence from XML.
print(texts, '\n')

# List: a list of 'term' in each text
print(terms, '\n')
# List: a list of 'category' in each text
print(categories, '\n')

# Dictionary: {term: number of terms founded in all texts}
print('term_positive\n', term_positive , '\n')
print('term_negative\n', term_negative , '\n')
print('term_neutral\n', term_neutral , '\n')
print('term_conflict\n', term_conflict , '\n')

# Dictionary: {category: number of category founded in all texts}
print('category_positive\n', category_positive , '\n')
print('category_negative\n', category_negative , '\n')
print('category_neutral\n', category_neutral , '\n')
print('category_conflict\n', category_conflict , '\n')
"""

"\n# List: a list of 'text' or a sentence from XML.\nprint(texts, '\n')\n\n# Dictionary: {term: number of terms founded in all texts}\nprint('term_positive\n', term_positive , '\n')\nprint('term_negative\n', term_negative , '\n')\nprint('term_neutral\n', term_neutral , '\n')\nprint('term_conflict\n', term_conflict , '\n')\n\n# Dictionary: {category: number of category founded in all texts}\nprint('category_positive\n', category_positive , '\n')\nprint('category_negative\n', category_negative , '\n')\nprint('category_neutral\n', category_neutral , '\n')\nprint('category_conflict\n', category_conflict , '\n')\n"

## Make a text to IOB format

Nichy is working on Step 3 about Regex to extract IOB as close as 'terms'.

Useful resource: https://www.nltk.org/book/ch07.html

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [18]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk.regexp import RegexpParser
from nltk.chunk import tree2conllstr
import nltk, string

insideWords = []
x = []

for t in texts:
    """ Step 1: Tokenization """
    tokenizedText = word_tokenize(t)
    #tokenizedText = list(filter(lambda token: token not in string.punctuation, tokenizedText))
    
    """ Step 2: Part-of-speech tagging """
    posText = pos_tag(tokenizedText)

    """ Step 3: Entity detection 
    This rule says that an NP chunk should be formed whenever the chunker finds an optional determiner (DT) followed by any number of adjectives (JJ) and then a noun (NN)
    """
    #grammar = "NP: {<DT>?<JJ>*<NN.*><IN>?<NN.*>*}"
    grammar = "NP: {<JJ|VBN>*<NN.*><IN>?<JJ|VBG|DT|IN>?<NN.*>*}"
    chunkParser = RegexpParser(grammar)
    chunkedText = chunkParser.parse(posText)
    
    """ Step 4: Relation detection. This relates to IOB format. """
    iobText = tree2conllstr(chunkedText)
    # x.append(iobText)
    x.append(getIOB(iobText))
    insideWord = getInsideOfIOB(iobText)
    insideWords.append(insideWord)
    
    

In [17]:
def getIOB(iobText):
    iobWords = iobText.split('\n')
    array = []
    for iob in iobWords:
      w = iob.split(" ")[0]
      tag = iob.split(" ")[2]
      array.append((w, tag))
    return array

In [20]:
x[:3]

[[('But', 'O'),
  ('the', 'O'),
  ('staff', 'B-NP'),
  ('was', 'O'),
  ('so', 'O'),
  ('horrible', 'O'),
  ('to', 'O'),
  ('us', 'O'),
  ('.', 'O')],
 [('To', 'O'),
  ('be', 'O'),
  ('completely', 'O'),
  ('fair', 'O'),
  (',', 'O'),
  ('the', 'O'),
  ('only', 'B-NP'),
  ('redeeming', 'I-NP'),
  ('factor', 'I-NP'),
  ('was', 'O'),
  ('the', 'O'),
  ('food', 'B-NP'),
  (',', 'O'),
  ('which', 'O'),
  ('was', 'O'),
  ('above', 'O'),
  ('average', 'B-NP'),
  (',', 'O'),
  ('but', 'O'),
  ('could', 'O'),
  ("n't", 'O'),
  ('make', 'O'),
  ('up', 'O'),
  ('for', 'O'),
  ('all', 'O'),
  ('the', 'O'),
  ('other', 'B-NP'),
  ('deficiencies', 'I-NP'),
  ('of', 'I-NP'),
  ('Teodora', 'I-NP'),
  ('.', 'O')],
 [('The', 'O'),
  ('food', 'B-NP'),
  ('is', 'O'),
  ('uniformly', 'O'),
  ('exceptional', 'O'),
  (',', 'O'),
  ('with', 'O'),
  ('a', 'O'),
  ('very', 'O'),
  ('capable', 'B-NP'),
  ('kitchen', 'I-NP'),
  ('which', 'O'),
  ('will', 'O'),
  ('proudly', 'O'),
  ('whip', 'O'),
  ('up', 'O'),
 

In [ ]:
import pandas as pd
df = pd.DataFrame(texts)
df['TextWithIOB'] = x


In [34]:
from google.colab import drive
drive.mount('/content/gdrive')
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/CS4248 NLP Project/iob.csv")

Mounted at /content/gdrive


In [35]:
!ls

gdrive	iob.csv  sample_data


Once we get IOB format, we find nouns from each IOB.
### 1. Get single-word-nouns

* Text:'Spice is great Thai food, love the inexpensive appetizers.'
* Given terms: ['Thai food', 'appetizers']
* IOB: ['Spice', 'Thai', 'food', 'love', 'appetizers']

### 2. Get multiple-word nouns

* Text: 'Spice is great Thai food, love the inexpensive appetizers.'
* Given terms: ['Thai food', 'appetizers']
* IOB: ['great Thai food', 'inexpensive appetizers']
      
You can see that the term is 'Thai food' but we get 'great Thai food'. This is because the code is set to include an adjective before a noun because some terms in other texts follow this pattern (e.g. term = 'green curry with vegetables'). If you fix 'great Thai food' to 'Thai food' then an issue will occurs with 'green curry with vegetatbles', it will become just 'curry with vegetatbles' which mismatches the given term. Therefore this is one of the issue to decide what pattern we have to follow.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
def getInsideOfIOB(iobText):
    iobWords = iobText.split('\n')
    insideWords = []
    
    #1. Extract single-word nouns 
    for i in range(len(iobWords)):
        word_and_iobs = iobWords[i].split(' ')
        word_and_iobs[0] = word_and_iobs[0].replace("'", "")
        if(word_and_iobs[2] == 'B-NP' and word_and_iobs[1] not in ['DT', 'JJ']):
            insideWords.append(word_and_iobs[0])
        elif(word_and_iobs[2] == 'I-NP' and word_and_iobs[1] in ['NN', 'NNS', 'NNP', 'NNPS']):
            insideWords.append(word_and_iobs[0])
    
    # 2.  multiple-word nouns 
    noun_string = ''
    for i in range(len(iobWords)):
        word_and_iobs = iobWords[i].split(' ')
        word_and_iobs[0] = word_and_iobs[0].replace("'", "")
        if(word_and_iobs[2] == 'B-NP' and word_and_iobs[1] != 'DT'):
            noun_string = word_and_iobs[0]
        elif(word_and_iobs[2] == 'I-NP'):
            noun_string = noun_string + ' ' + word_and_iobs[0]
        elif(word_and_iobs[2] == 'O' and noun_string != '') :
            noun_string = removeSpaceBeforeNoun(noun_string)
            if ' ' in noun_string:
                insideWords.append(noun_string)
            noun_string = ''
        
        if(i == len(iobWords)-1 and noun_string != ''): # End of the text
            noun_string = removeSpaceBeforeNoun(noun_string)
            if ' ' in noun_string:
                insideWords.append(noun_string)
    return insideWords    

def removeSpaceBeforeNoun(noun):
    if(noun[:1] == ' '):
        noun = noun[1:]
    return noun



# Number of texts that keywords from IOB match to all terms

In [ ]:
import numpy as np
total_score = 0
for i in range(3044):
    score = checkTermInIOB(terms[i], insideWords[i])
    if(score == 1.0):
        total_score = total_score + 1
print('Number of texts that IOBs match terms:', total_score, 'out of', 3044)
print('Percentage:', np.round(total_score/3044, 4))

Number of texts that IOBs match terms: 2563 out of 3044
Percentage: 0.842


# Text that some of its IOB is still cannot exactly match 'terms'

In [ ]:
def checkTermInIOB(terms, iobs):
    count = 0
    for t in terms:
        if t in iobs:
            count = count + 1
    #print(terms, iobs, count, len(terms))
    if(count == len(terms)):
        return 1.0
    elif(len(terms) == 0 ):
        return 1.0
    else:
        return 0.0

for i in range(3044):
    score = checkTermInIOB(terms[i], insideWords[i])
    not_in_iob_words = []
    for t in terms[i]:
        if t not in insideWords[i]: 
            not_in_iob_word = t
            not_in_iob_words.append(not_in_iob_word)
    #score = 0
    if(score != 1):
        print('ID:', i)
        #print('Terms not in IOB       :', not_in_iob_words)
        print('Terms      :', terms[i])
        print('IOB word   :', insideWords[i],'\n')



In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

# Tim's part

Place holder for Tim's pre-processing

In [ ]:
!pip install contractions
!pip install SpellChecker

In [ ]:
import xml.etree.ElementTree as ET
from enum import Enum
import pandas as pd

restaurant = ET.parse('Restaurants_Train.xml')
sentences = restaurant.getroot()
class Cols(Enum):
    ID = 1
    Text = 2
    AspectTerms = 3
    PreProcessed = 4
data = {Cols.ID:[],Cols.Text:[],Cols.AspectTerms:[]}

for sentence in sentences:
    term_list = []
    data[Cols.ID].append(sentence.attrib['id'])
    for taa in sentence: # taa is the level of <text>, <aspectTerms>, <aspectCategories>
        if(taa.tag == 'text'):
            data[Cols.Text].append(taa.text)
            #print(sentence.tag, taa.tag, ':', taa.text)
        elif(taa.tag == 'aspectTerms'): 
           for aspectTerms in taa:
                #print(sentence.tag, taa.tag, ':', aspectTerms.attrib) # attrib is dictionary
                term = aspectTerms.attrib['term']
                polarity = aspectTerms.attrib['polarity']
                idx_from = aspectTerms.attrib['from']
                idx_to = aspectTerms.attrib['to']
                term_list.append((term,polarity,idx_from,idx_to))
    data[Cols.AspectTerms].append(term_list)

df=pd.DataFrame.from_dict(data)
print(df)

# TODO: need to check how does it work in google drive
df.to_csv("pre_processed.csv")

FileNotFoundError: ignored

In [ ]:
import contractions
# all of the below act on a single text

def fix_contractions(text):
    return contractions.fix(text)

# return [(word, POS)......]
def get_pos(text)
    return nltk.posTag(nltk.word_tokenize(text))

def basic_pre_process(text)
    return get_pos(fix_contractions(text))

df[Cols.PreProcessed] = df[Cols.Text].map(basic_pre_process)


pos_replace = {"CD":"<NUM>","NNP":"<PNOUN>","NNPS":"<PNOUN>"}

#3 required imput [(word, POS)......]
def replace(x):
    ls = []
    for word,tag in x:
        if tag in pos_replace:
            ls.append((pos_replace[tag],tag))
        # might not be useful here
        # elif any(char.isdigit() for char in word):
        #         ls.append(("<NUM>","CD"))
        else:
            ls.append((word,tag))
    return ls

def mispell_check(text):
    mispelled = [i[0] for i in text]
    return mispelled

#TODO: investigating jamspell
#https://github.com/bakwc/JamSpell
#https://stackoverflow.com/questions/43984158/are-there-any-auto-correct-auto-complete-libraries-for-python
def mispell_fix(text):
    return text


## Dependency Parsing


In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
text1 = "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not."
doc = nlp(text1)
for token in doc:
    print(token.text, token.tag_, token.head.text, token.dep_)
displacy.render(nlp(text1),jupyter=True)